In [1]:
from keras.models import (
    Model,
    model_from_json
)
import json
from scipy import spatial

# this will probably need to me moved somwhere else...
def load_model(path):
    # load json and create model
    json_file = open('%s.json' % path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("%s.h5" % path)
    print("Loaded model from disk")
    return loaded_model

def get_similarity(vec1, vec2):
    return 1 - spatial.distance.cosine(vec1, vec2)

mapping = json.loads(open('../data/mapping.json').read())
reverse_mapping = json.loads(open('../data/reverse_mapping.json').read())

Using TensorFlow backend.


In [2]:
vectorizer = load_model('../data/model/word_embedder')

Loaded model from disk


In [3]:
def vectorize_word(word):
    return vectorizer.predict([[mapping[word]]])

def get_word_similarity(word1, word2):
    return get_similarity(
        vectorize_word(word1),
        vectorize_word(word2)
    )
    

In [4]:
import numpy as np
from emoji import UNICODE_EMOJI
valid_emojis = [e for e in UNICODE_EMOJI if e in mapping.keys()]
vectors = []
for e in valid_emojis:
    vectors.append(vectorize_word(e)[0])
vectors = np.asarray(vectors)
vectors = vectors.reshape((len(valid_emojis), 300))

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
reduced = pca.fit_transform(vectors)

In [7]:
def get_most_similar_emoji(emoji):
    return sorted(valid_emojis, key=lambda x: -get_word_similarity(emoji, x))[:10]

In [27]:
get_most_similar_emoji('😤')

['😤', '😡', '😐', '\U0001f92f', '\U0001f928', '👉', '😭', '👈', '\U0001f92c', '🚫']

In [24]:
import plotly.express as px
import pandas as pd
def plot(emoji, n=50):
    data = pd.DataFrame(reduced)
    data.columns = ['x', 'y']
    data['text'] = valid_emojis
    data['sortval'] = data.text.apply(lambda x: get_word_similarity(x, emoji))
    data = data.sort_values('sortval', ascending=False)
    fig = px.scatter(data.head(n), x='x', y='y', text='text')
    fig.show()

In [26]:
plot('😊')
plot('😡')